In [1]:
import pymc as pm
import numpy as np

grid_size = 7
n_regions = grid_size**2

# make random consistent data
np.random.seed(1)

# Expected counts - keep it constant for simplicity
E = np.ones(n_regions) * 30

# Simulated observed counts
y_obs = np.random.poisson(lam=E)

# Assuming W is an adjacency matrix and D is the diagonal degree matrix
rho = 0.99  # Spatial correlation parameter (adjust as needed)

# Define adjacency matrix for grid
W = np.zeros((n_regions, n_regions))

for i in range(n_regions):
    if i % grid_size > 0:
        W[i, i - 1] = 1
        W[i - 1, i] = 1
    if i // grid_size > 0:
        W[i, i - grid_size] = 1
        W[i - grid_size, i] = 1

# Define precision matrix for CAR model
D = np.diag(W.sum(axis=1))  # Diagonal degree matrix
Q = D - rho * W  # Intrinsic CAR precision matrix

with pm.Model() as poisson_model_N:
    # Prior for random effect (Normal)
    vN_prior = pm.Normal("vN", mu=0.5, sigma=0.125, shape=n_regions)

    # Prior for spatial effect (CAR)
    tau = pm.Gamma("tau", alpha=0.5, beta=0.5)
    uN_prior = pm.MvNormal("uN", mu=np.zeros(n_regions), tau=tau * Q, shape=n_regions)

    # Define risk parameter
    thetaN = pm.Deterministic("thetaN", pm.math.exp(pm.math.log(E) + uN_prior + vN_prior))

    # Define Poisson likelihood
    y = pm.Poisson("y", mu=thetaN, observed=y_obs)

    trace1 = pm.sample(1000, tune=1000, target_accept=0.9,return_inferencedata=True, log_likelihood=True)



Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [vN, tau, uN]


/Users/h6x/anaconda3/envs/pymc_env/lib/python3.12/site-packages/rich/live.py:231: UserWarning: install "ipywidgets"
for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 12 seconds.


In [2]:
with pm.Model() as poisson_model_2:
    # Prior for random effect (Normal)
    vN_prior = pm.Normal("vN", mu=0.7, sigma=0.225, shape=n_regions)

    # Prior for spatial effect (CAR)
    tau = pm.Gamma("tau", alpha=0.7, beta=0.7)
    uN_prior = pm.MvNormal("uN", mu=np.zeros(n_regions), tau=tau * Q, shape=n_regions)

    # Define risk parameter
    thetaN = pm.Deterministic("thetaN", pm.math.exp(pm.math.log(E) + uN_prior + vN_prior))

    # Define Poisson likelihood
    y = pm.Poisson("y", mu=thetaN, observed=y_obs)

    trace2 = pm.sample(1000, tune=1000, target_accept=0.9,return_inferencedata=True, log_likelihood=True)

Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [vN, tau, uN]


/Users/h6x/anaconda3/envs/pymc_env/lib/python3.12/site-packages/rich/live.py:231: UserWarning: install "ipywidgets"
for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 34 seconds.


In [3]:
import arviz as az

In [4]:
az.compare(
    {'m1':trace1,'m2':trace2},
    scale='deviance',
    ic='waic',
    method="BB-pseudo-BMA"
)

TypeError: Encountered error in ELPD computation of compare.

In [ ]:
compare_dict = {"Model 1 (v estimated)": trace1, "Model 2 (v fixed)": trace2}
comp = az.compare(compare_dict, method="WAIC")
print("\nModel Comparison (WAIC):")
print(comp)

# Optionally, plot the comparison
# az.plot_compare(comp)
# plt.show()

In [ ]:
# -----------------------------
# 5. Compare Models via WAIC (for reference)
# -----------------------------
compare_dict = {"Model 1 (v estimated)": trace1, "Model 2 (v fixed)": trace2}
waic_comp = az.compare(compare_dict, method="WAIC")
print("\nModel Comparison (WAIC):")
print(waic_comp)
# az.plot_compare(waic_comp)
# plt.show()